In [62]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
pd.set_option('display.max_columns', 200)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
query = """
SELECT * 
FROM nppes as np
join npi as n
on np.NPI = n.from_npi
WHERE np.CBSA = 34980

"""
with sqlite3.connect('../data/npidata.sqlite') as db:
    nppes_nashville = pd.read_sql(query, db)

In [7]:
db = sqlite3.connect('../data/npidata.sqlite')

query = '''
SELECT *
FROM npi
'''

hop_teams_npi = pd.read_sql(query,db)
db.close()

In [8]:
db = sqlite3.connect('../data/npidata.sqlite')

query = '''
SELECT *
FROM nppes
'''

hop_teams_nppes = pd.read_sql(query,db)
db.close()

In [53]:
hop_teams_nppes

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1679576722,1.0,None,WIEBE,DAVID,A,None,None,M.D.,3500 CENTRAL AVE,None,KEARNEY,NE,688472944.0,207X00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207X00000X,Orthopaedic Surgery,68847,28260
1,1588667638,1.0,None,PILCHER,WILLIAM,C,DR.,None,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X,Y,207RC0000X,N,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207RC0000X,Internal Medicine,32204,27260
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",None,None,None,None,None,None,3418 VILLAGE DR,None,FAYETTEVILLE,NC,283044552.0,251G00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,251G00000X,"Hospice Care, Community Based",28304,22180
3,1306849450,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,000.0,0
4,1215930367,1.0,None,GRESSOT,LAURENT,None,DR.,None,M.D.,17323 RED OAK DR,None,HOUSTON,TX,770901243.0,174400000X,N,207RH0003X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207RH0003X,Internal Medicine,77090,26420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8989945,1942064936,1.0,None,BAROCIO,VALERIE,None,None,None,None,600 N ARROWHEAD AVE,None,SAN BERNARDINO,CA,924011164.0,172V00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,172V00000X,Community Health Worker,92401,40140
8989946,1851155840,1.0,None,SHUFFETT,WILLIAM,THOMAS,MR.,None,None,3934 DIXIE HWY STE 201,None,LOUISVILLE,KY,402164163.0,101Y00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,

In [9]:
# Subset data to facilities and providers
all_facilities = hop_teams_nppes[hop_teams_nppes['Entity Type Code'] == 2.0]
all_providers = hop_teams_nppes[hop_teams_nppes['Entity Type Code'] == 1.0]
# Filling all nulls with blank string
all_providers = all_providers.fillna('')
all_providers['Provider Full Name'] = \
all_providers['Provider Name Prefix Text'] + \
all_providers['Provider First Name'] + ' ' + \
all_providers['Provider Middle Name'] + ' ' + \
all_providers['Provider Last Name (Legal Name)'] + ' ' + \
all_providers['Provider Name Suffix Text'] + ' ' + \
all_providers['Provider Credential Text']
# Creating a dictionary for all the Business NPI's
npi_to_business = dict(zip(all_facilities['NPI'],all_facilities['Provider Organization Name (Legal Business Name)']))
# Creating a dictionary for all the Provider NPI's
npi_to_provider = dict(zip(all_providers['NPI'],all_providers['Provider Full Name']))
# Combining the Business and Provider dictionary
all_npi_classification = npi_to_business | npi_to_provider

In [126]:
# Create dataframe with the NPI's and their respective provider
npi_to_name = pd.DataFrame({'npi':list(all_npi_classification.keys()),
                    'name':list(all_npi_classification.values())})

In [127]:
npi_to_name

,npi,name
0,1497758544,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC"
1,1023011178,COLLABRIA CARE
2,1023011079,"ADVANTAGE HOME HEALTH CARE, INC."
3,1487657433,"PEKIN MRI, LLC"
4,1740283795,EASTERN STAR MASONIC HOME
...,...,...
7911245,1942064936,VALERIE BAROCIO
7911246,1851155840,MR.WILLIAM THOMAS SHUFFETT
7911247,1760246755,CHRISTOPHER MCASTOCKER
7911248,1679337661,COURTNEY A HELTON APRN


In [12]:
competitor_hospitals = ['ENDOSCOPY CENTER OF ST THOMAS LLC',
       'SAINT THOMAS CAMPUS SURGICARE LP',
       'SAINT THOMAS OUTPATIENT NEUROSURGICAL CENTER',
       'SAINT THOMAS MEDICAL PARTNERS', 'SAINT THOMAS WEST HOSPITAL',
       'SAINT THOMAS HICKMAN HOSPITAL', 'SAINT THOMAS RX LLC',
       'SAINT THOMAS RUTHERFORD HOSPITAL',
       'SAINT THOMAS HOME HEALTH, INC.', 'SAINT THOMAS CLINIC PLLC',
       'SAINT THOMAS HEALTH', 'SAINT THOMAS HIGHLANDS HOSPITAL, LLC',
       'SAINT THOMAS RIVER PARK HOSPITAL, LLC',
       'SAINT THOMAS STONES RIVER HOSPITAL, LLC',
       'SAINT THOMAS DEKALB HOSPITAL, LLC', 'SAINT THOMAS CARE PLUS LLC',
                        'CENTENNIAL SURGERY CENTER LP',
       'COLUMBIA MEDICAL GROUP-CENTENNIAL INC',
       'CENTENNIAL SURGICAL ASSOCIATES LLC', 'CENTENNIAL HEART LLC',
       'CENTENNIAL PSYCHIATRIC ASSOCIATES, LLC',
       'CENTENNIAL NEUROSCIENCE, LLC', 'CENTENNIAL SURGICAL CLINIC, LLC',
       'TRISTAR MEDICAL GROUP - CENTENNIAL PRIMARY CARE, LLC',
       'CENTENNIAL HOSPITALISTS, LLC', 'CENTENNIAL WOMENS GROUP, LLC', 'TRISTAR CARDIOVASCULAR SURGERY LLC',
       'TRISTAR MEDICAL GROUP - CENTENNIAL PRIMARY CARE, LLC',
       'TRISTAR RADIATION ONCOLOGY, LLC',
       'TRISTAR GYNECOLOGY ONCOLOGY, LLC',
       'TRISTAR JOINT REPLACEMENT INSTITUTE, LLC',
       'TRISTAR BONE MARROW TRANSPLANT, LLC', 'TRISTAR FAMILY CARE, LLC',
                      'NASHVILLE GENERAL HOSPITAL' ]

tri_star_hospitals = ['TRISTAR CARDIOVASCULAR SURGERY LLC',
       'TRISTAR MEDICAL GROUP - CENTENNIAL PRIMARY CARE, LLC',
       'TRISTAR RADIATION ONCOLOGY, LLC',
       'TRISTAR GYNECOLOGY ONCOLOGY, LLC',
       'TRISTAR JOINT REPLACEMENT INSTITUTE, LLC',
       'TRISTAR BONE MARROW TRANSPLANT, LLC', 'TRISTAR FAMILY CARE, LLC',
       'TRISTAR MEDICAL GROUP - LEGACY HEALTH, LLC']

vanderbilt_hospitals = ['VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP',
       'VANDERBILT UNIVERSITY MEDICAL CENTER',
       'VANDERBILT IMAGING SERVICES, LLC',
       'VANDERBILT HEALTH AND WILLIAMSON MEDICAL CENTER CLINICS AND SERVICES,',
       'VANDERBILT HOME CARE SERVICES, LLC',
       'VANDERBILT MAURY RADIATION ONCOLOGY, LLC']

In [132]:
to_npi_to_name = npi_to_name.rename(columns = {'npi' : 'to_npi'})
npi_to_name = npi_to_name.rename(columns = {'npi': 'NPI', 'name': 'from_provider'})

In [129]:
nppes_nashville = nppes_nashville.merge(right = to_npi_to_name, how = 'left', on = 'to_npi')
nppes_nashville.head()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,name_x,name_y
0,1134122187,1.0,None,RUDNICKE,CHERYL,DENISE,MRS.,None,CRNP,250 25TH AVE N,STE 412,NASHVILLE,TN,372031632.0,363L00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,363L00000X,Nurse Practitioner,37203,34980,1134122187,1568464873,52,52,21.327,21.192,DICKSON MEDICAL ASSOCIATES PC,DICKSON MEDICAL ASSOCIATES PC
1,1134122187,1.0,None,RUDNICKE,CHERYL,DENISE,MRS.,None,CRNP,250 25TH AVE N,STE 412,NASHVILLE,TN,372031632.0,363L00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,363L00000X,Nurse Practitioner,37203,34980,1134122187,1699713164,53,53,0.000,0.000,NHC HEALTHCARE-DICKSON LLC,NHC HEALTHCARE-DICKSON LLC
2,1134122187,1.0,None,RUDNICKE,CHERYL,DENISE,MRS.,None,CRNP,250 25TH AVE N,STE 412,NASHVILLE,TN,372031632.0,363L00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,363L00000X,Nurse Practitioner,37203,34980,1134122187,1750761060,53,53,0.000,0.000,CURANA HEALTH OF TENNESSEE LLC,CURANA HEALTH OF TENNESSEE LLC
3,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,370551303.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1750384780,1003863580,101,129,32.000,38.731,"ASSOCIATED PATHOLOGISTS, LLC","ASSOCIATED PATHOLOGISTS, LLC"
4,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,370551303.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1750384780,1013175215,31,61,4.639,25.993,"LHC HOMECARE OF TENNESSEE, LLC","LHC HOMECARE OF TENNESSEE, LLC"


In [164]:
nppes_nashville = nppes_nashville.merge(right = npi_to_name, how = 'left', on ='NPI')

In [23]:
refer_to_competitor = nppes_nashville[nppes_nashville['name'].isin(competitor_hospitals)]

In [34]:
refer_to_competitor.groupby('Classification')['patient_count'] \
.sum() \
.reset_index() \
.sort_values(by = 'patient_count', ascending = False) \
.reset_index(drop = True)

,Classification,patient_count
0,Internal Medicine,644157
1,Radiology,342013
2,General Acute Care Hospital,138066
3,Emergency Medicine,101268
4,Anesthesiology,87669
...,...,...
65,Psychiatric Hospital,47
66,Dentist,46
67,Public Health or Welfare,41
68,Phlebology,40


In [38]:
refer_to_vanderbilt = nppes_nashville[nppes_nashville['name'].isin(vanderbilt_hospitals)]

In [39]:
refer_to_vanderbilt.groupby('Classification')['patient_count'] \
.sum() \
.reset_index() \
.sort_values(by = 'patient_count', ascending = False) \
.reset_index(drop = True)

,Classification,patient_count
0,Internal Medicine,545765
1,Radiology,242101
2,General Acute Care Hospital,126961
3,Nurse Practitioner,93262
4,Family Medicine,56250
...,...,...
62,Medical Genetics,108
63,Psychiatric Unit,101
64,Rehabilitation Unit,100
65,Licensed Practical Nurse,35


In [86]:
refer_to_tristar = nppes_nashville[nppes_nashville['name'].isin(tri_star_hospitals)]

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,name
380,1275536468,1.0,None,ASAD,MUHAMMAD,None,DR.,None,MD,353 NEW SHACKLE ISLAND RD,SUITE 300 C,HENDERSONVILLE,TN,370752379.0,207RI0011X,N,207RC0000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207RC0000X,Internal Medicine,37075,34980,1275536468,1770851552,45,51,32.922,67.310,TRISTAR CARDIOVASCULAR SURGERY LLC
1568,1568464873,2.0,DICKSON MEDICAL ASSOCIATES PC,None,None,None,None,None,None,113 HIGHWAY 70 E,None,DICKSON,TN,370552080.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1568464873,1164834057,68,88,46.807,61.499,"TRISTAR JOINT REPLACEMENT INSTITUTE, LLC"
1759,1568464873,2.0,DICKSON MEDICAL ASSOCIATES PC,None,None,None,None,None,None,113 HIGHWAY 70 E,None,DICKSON,TN,370552080.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1568464873,1609326040,42,65,33.154,37.904,"TRISTAR FAMILY CARE, LLC"
1824,1568464873,2.0,DICKSON MEDICAL ASSOCIATES PC,None,None,None,None,None,None,113 HIGHWAY 70 E,None,DICKSON,TN,370552080.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1568464873,1770851552,46,63,12.810,20.686,TRISTAR CARDIOVASCULAR SURGERY LLC
2703,1215932413,2.0,"ANESTHESIA MEDICAL GROUP, PC",None,None,None,None,None,None,110 29TH AVE N,STE 301,NASHVILLE,TN,37203.0,207R00000X,N,207RP1001X,N,2081P2900X,N,208VP0000X,N,367500000X,N,207L00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207L00000X,Anesthesiology,37203,34980,1215932413,1003152208,274,324,17.222,42.241,"TRISTAR GYNECOLOGY ONCOLOGY, LLC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [43]:
refer_to_competitor.merge(right = refer_to_vanderbilt, how = 'left', on = 'NPI' )

,NPI,Entity Type Code_x,Provider Organization Name (Legal Business Name)_x,Provider Last Name (Legal Name)_x,Provider First Name_x,Provider Middle Name_x,Provider Name Prefix Text_x,Provider Name Suffix Text_x,Provider Credential Text_x,Provider First Line Business Practice Location Address_x,Provider Second Line Business Practice Location Address_x,Provider Business Practice Location Address City Name_x,Provider Business Practice Location Address State Name_x,Provider Business Practice Location Address Postal Code_x,Healthcare Provider Taxonomy Code_1_x,Healthcare Provider Primary Taxonomy Switch_1_x,Healthcare Provider Taxonomy Code_2_x,Healthcare Provider Primary Taxonomy Switch_2_x,Healthcare Provider Taxonomy Code_3_x,Healthcare Provider Primary Taxonomy Switch_3_x,Healthcare Provider Taxonomy Code_4_x,Healthcare Provider Primary Taxonomy Switch_4_x,Healthcare Provider Taxonomy Code_5_x,Healthcare Provider Primary Taxonomy Switch_5_x,Healthcare Provider Taxonomy Code_6_x,Healthcare Provider Primary Taxonomy Switch_6_x,Healthcare Provider Taxonomy Code_7_x,Healthcare Provider Primary Taxonomy Switch_7_x,Healthcare Provider Taxonomy Code_8_x,Healthcare Provider Primary Taxonomy Switch_8_x,Healthcare Provider Taxonomy Code_9_x,Healthcare Provider Primary Taxonomy Switch_9_x,Healthcare Provider Taxonomy Code_10_x,Healthcare Provider Primary Taxonomy Switch_10_x,Healthcare Provider Taxonomy Code_11_x,Healthcare Provider Primary Taxonomy Switch_11_x,Healthcare Provider Taxonomy Code_12_x,Healthcare Provider Primary Taxonomy Switch_12_x,Healthcare Provider Taxonomy Code_13_x,Healthcare Provider Primary Taxonomy Switch_13_x,Healthcare Provider Taxonomy Code_14_x,Healthcare Provider Primary Taxonomy Switch_14_x,Healthcare Provider Taxonomy Code_15_x,Healthcare Provider Primary Taxonomy Switch_15_x,Taxonomy_Code_x,Classification_x,zipcodes_x,CBSA_x,from_npi_x,to_npi_x,patient_count_x,transaction_count_x,average_day_wait_x,std_day_wait_x,name_x,Entity Type Code_y,Provider Organization Name (Legal Business Name)_y,Provider Last Name (Legal Name)_y,Provider First Name_y,Provider Middle Name_y,Provider Name Prefix Text_y,Provider Name Suffix Text_y,Provider Credential Text_y,Provider First Line Business Practice Location Address_y,Provider Second Line Business Practice Location Address_y,Provider Business Practice Location Address City Name_y,Provider Business Practice Location Address State Name_y,Provider Business Practice Location Address Postal Code_y,Healthcare Provider Taxonomy Code_1_y,Healthcare Provider Primary Taxonomy Switch_1_y,Healthcare Provider Taxonomy Code_2_y,Healthcare Provider Primary Taxonomy Switch_2_y,Healthcare Provider Taxonomy Code_3_y,Healthcare Provider Primary Taxonomy Switch_3_y,Healthcare Provider Taxonomy Code_4_y,Healthcare Provider Primary Taxonomy Switch_4_y,Healthcare Provider Taxonomy Code_5_y,Healthcare Provider Primary Taxonomy Switch_5_y,Healthcare Provider Taxonomy Code_6_y,Healthcare Provider Primary Taxonomy Switch_6_y,Healthcare Provider Taxonomy Code_7_y,Healthcare Provider Primary Taxonomy Switch_7_y,Healthcare Provider Taxonomy Code_8_y,Healthcare Provider Primary Taxonomy Switch_8_y,Healthcare Provider Taxonomy Code_9_y,Healthcare Provider Primary Taxonomy Switch_9_y,Healthcare Provider Taxonomy Code_10_y,Healthcare Provider Primary Taxonomy Switch_10_y,Healthcare Provider Taxonomy Code_11_y,Healthcare Provider Primary Taxonomy Switch_11_y,Healthcare Provider Taxonomy Code_12_y,Healthcare Provider Primary Taxonomy Switch_12_y,Healthcare Provider Taxonomy Code_13_y,Healthcare Provider Primary Taxonomy Switch_13_y,Healthcare Provider Taxonomy Code_14_y,Healthcare Provider Primary Taxonomy Switch_14_y,Healthcare Provider Taxonomy Code_15_y,Healthcare Provider Primary Taxonomy Switch_15_y,Taxonomy_Code_y,Classification_y,zipcodes_y,CBSA_y,from_npi_y,to_npi_y,patient_count_y,transaction_count_y,average_day_wait_y,std_day_wait_y,name_y
0,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHUR

In [40]:
refer_to_competitor

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,name
12,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,370551303.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1750384780,1245393057,39,58,42.172,45.896,CENTENNIAL HEART LLC
16,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,370551303.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1750384780,1437194669,37,67,36.672,42.602,SAINT THOMAS MEDICAL PARTNERS
41,1073516001,1.0,None,ROSS,DAVID,L,DR.,None,MD,127 CRESTVIEW PARK DR,None,DICKSON,TN,370552850.0,207R00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207R00000X,Internal Medicine,37055,34980,1073516001,1245393057,40,63,34.810,37.096,CENTENNIAL HEART LLC
45,1073516001,1.0,None,ROSS,DAVID,L,DR.,None,MD,127 CRESTVIEW PARK DR,None,DICKSON,TN,370552850.0,207R00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207R00000X,Internal Medicine,37055,34980,1073516001,1437194669,62,106,37.047,50.358,SAINT THOMAS MEDICAL PARTNERS
52,1073516001,1.0,None,ROSS,DAVID,L,DR.,None,MD,127 CRESTVIEW PARK DR,None,DICKSON,TN,370552850.0,207R00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207R00000X,Internal Medicine,37055,34980,1073516001,1629025648,34,54,36.685,45.278,SAINT THOMAS WEST HOSPITAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237679,1558862961,1.0,None,JACKSON,NOELLE,MORGAN,None,None,None,4230 HARDING PIKE STE 410,None,NASHVILLE,TN,372052013.0,163

In [41]:

refer_to_vanderbilt

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,name
8,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,370551303.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1750384780,1104202761,36,68,38.456,43.634,VANDERBILT UNIVERSITY MEDICAL CENTER
15,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,370551303.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1750384780,1396882205,30,54,35.759,39.998,VANDERBILT UNIVERSITY MEDICAL CENTER
36,1073516001,1.0,None,ROSS,DAVID,L,DR.,None,MD,127 CRESTVIEW PARK DR,None,DICKSON,TN,370552850.0,207R00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207R00000X,Internal Medicine,37055,34980,1073516001,1104202761,74,135,31.444,37.397,VANDERBILT UNIVERSITY MEDICAL CENTER
44,1073516001,1.0,None,ROSS,DAVID,L,DR.,None,MD,127 CRESTVIEW PARK DR,None,DICKSON,TN,370552850.0,207R00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207R00000X,Internal Medicine,37055,34980,1073516001,1396882205,60,114,28.395,31.253,VANDERBILT UNIVERSITY MEDICAL CENTER
78,1780687830,1.0,None,MANI,VENK,None,DR.,None,MD,127 CRESTVIEW PARK DR,None,DICKSON,TN,370552850.0,207ZC0500X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207ZC0500X,Pathology,37055,34980,1780687830,1104202761,178,258,34.562,48.274,VANDERBILT UNIVERSITY MEDICAL CENTER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237617,1982104345,1.0,None,BARRENTINE,PAUL,BRENT,None,None,NP,1211 MEDI

In [72]:
merged_competitors_vandy = refer_to_competitor.merge(
right = refer_to_vanderbilt,
how = 'left',
on = 'NPI')

In [76]:
# Providers that send only to vanderbilt competitors
sent_to_vandy_comp = merged_competitors_vandy[merged_competitors_vandy['name_y'].isna()]
sent_to_vandy_comp

,NPI,Entity Type Code_x,Provider Organization Name (Legal Business Name)_x,Provider Last Name (Legal Name)_x,Provider First Name_x,Provider Middle Name_x,Provider Name Prefix Text_x,Provider Name Suffix Text_x,Provider Credential Text_x,Provider First Line Business Practice Location Address_x,Provider Second Line Business Practice Location Address_x,Provider Business Practice Location Address City Name_x,Provider Business Practice Location Address State Name_x,Provider Business Practice Location Address Postal Code_x,Healthcare Provider Taxonomy Code_1_x,Healthcare Provider Primary Taxonomy Switch_1_x,Healthcare Provider Taxonomy Code_2_x,Healthcare Provider Primary Taxonomy Switch_2_x,Healthcare Provider Taxonomy Code_3_x,Healthcare Provider Primary Taxonomy Switch_3_x,Healthcare Provider Taxonomy Code_4_x,Healthcare Provider Primary Taxonomy Switch_4_x,Healthcare Provider Taxonomy Code_5_x,Healthcare Provider Primary Taxonomy Switch_5_x,Healthcare Provider Taxonomy Code_6_x,Healthcare Provider Primary Taxonomy Switch_6_x,Healthcare Provider Taxonomy Code_7_x,Healthcare Provider Primary Taxonomy Switch_7_x,Healthcare Provider Taxonomy Code_8_x,Healthcare Provider Primary Taxonomy Switch_8_x,Healthcare Provider Taxonomy Code_9_x,Healthcare Provider Primary Taxonomy Switch_9_x,Healthcare Provider Taxonomy Code_10_x,Healthcare Provider Primary Taxonomy Switch_10_x,Healthcare Provider Taxonomy Code_11_x,Healthcare Provider Primary Taxonomy Switch_11_x,Healthcare Provider Taxonomy Code_12_x,Healthcare Provider Primary Taxonomy Switch_12_x,Healthcare Provider Taxonomy Code_13_x,Healthcare Provider Primary Taxonomy Switch_13_x,Healthcare Provider Taxonomy Code_14_x,Healthcare Provider Primary Taxonomy Switch_14_x,Healthcare Provider Taxonomy Code_15_x,Healthcare Provider Primary Taxonomy Switch_15_x,Taxonomy_Code_x,Classification_x,zipcodes_x,CBSA_x,from_npi_x,to_npi_x,patient_count_x,transaction_count_x,average_day_wait_x,std_day_wait_x,name_x,Entity Type Code_y,Provider Organization Name (Legal Business Name)_y,Provider Last Name (Legal Name)_y,Provider First Name_y,Provider Middle Name_y,Provider Name Prefix Text_y,Provider Name Suffix Text_y,Provider Credential Text_y,Provider First Line Business Practice Location Address_y,Provider Second Line Business Practice Location Address_y,Provider Business Practice Location Address City Name_y,Provider Business Practice Location Address State Name_y,Provider Business Practice Location Address Postal Code_y,Healthcare Provider Taxonomy Code_1_y,Healthcare Provider Primary Taxonomy Switch_1_y,Healthcare Provider Taxonomy Code_2_y,Healthcare Provider Primary Taxonomy Switch_2_y,Healthcare Provider Taxonomy Code_3_y,Healthcare Provider Primary Taxonomy Switch_3_y,Healthcare Provider Taxonomy Code_4_y,Healthcare Provider Primary Taxonomy Switch_4_y,Healthcare Provider Taxonomy Code_5_y,Healthcare Provider Primary Taxonomy Switch_5_y,Healthcare Provider Taxonomy Code_6_y,Healthcare Provider Primary Taxonomy Switch_6_y,Healthcare Provider Taxonomy Code_7_y,Healthcare Provider Primary Taxonomy Switch_7_y,Healthcare Provider Taxonomy Code_8_y,Healthcare Provider Primary Taxonomy Switch_8_y,Healthcare Provider Taxonomy Code_9_y,Healthcare Provider Primary Taxonomy Switch_9_y,Healthcare Provider Taxonomy Code_10_y,Healthcare Provider Primary Taxonomy Switch_10_y,Healthcare Provider Taxonomy Code_11_y,Healthcare Provider Primary Taxonomy Switch_11_y,Healthcare Provider Taxonomy Code_12_y,Healthcare Provider Primary Taxonomy Switch_12_y,Healthcare Provider Taxonomy Code_13_y,Healthcare Provider Primary Taxonomy Switch_13_y,Healthcare Provider Taxonomy Code_14_y,Healthcare Provider Primary Taxonomy Switch_14_y,Healthcare Provider Taxonomy Code_15_y,Healthcare Provider Primary Taxonomy Switch_15_y,Taxonomy_Code_y,Classification_y,zipcodes_y,CBSA_y,from_npi_y,to_npi_y,patient_count_y,transaction_count_y,average_day_wait_y,std_day_wait_y,name_y
28,1992708911,1.0,None,RUSSELL,NATHAN,W,None,None,FNP,166 E

In [93]:
# Which providers are sending only to competitor hospitals
sent_to_vandy_comp.groupby('Provider Organization Name (Legal Business Name)_x')['patient_count_x'] \
.sum() \
.reset_index() \
.sort_values(by = 'patient_count_x', ascending = False)

,Provider Organization Name (Legal Business Name)_x,patient_count_x
71,"HCA HEALTH SERVICES OF TENNESSEE, INC.",22200
90,"LAB PLUS, LLC",19611
0,ACLA PC,14609
116,MIDDLE TENNESSEE IMAGING LLC,10570
123,"MURFREESBORO ANESTHESIA GROUP, P.A.",5040
...,...,...
199,STEEL FAMILY MEDICINE,22
227,TOTAL RENAL CARE INC,20
237,"VALLEY MEDICAL CENTER, PLLC",18
75,"HIDDEN ACRES HEALTHCARE, LLC",17


In [87]:
# Creating dataframe of all providers that sent to Tristar or Vanderbilt
merged_tristar_vandy = refer_to_tristar.merge(
right = refer_to_vanderbilt,
how = 'left',
on = 'NPI')

In [88]:
# Remove all providers that sent to Vanderbilt, so we can have only providers that sent to Tri-Star exclusively
sent_to_vandy_tri_star = merged_tristar_vandy[merged_tristar_vandy['name_y'].isna()]
sent_to_vandy_tri_star

,NPI,Entity Type Code_x,Provider Organization Name (Legal Business Name)_x,Provider Last Name (Legal Name)_x,Provider First Name_x,Provider Middle Name_x,Provider Name Prefix Text_x,Provider Name Suffix Text_x,Provider Credential Text_x,Provider First Line Business Practice Location Address_x,Provider Second Line Business Practice Location Address_x,Provider Business Practice Location Address City Name_x,Provider Business Practice Location Address State Name_x,Provider Business Practice Location Address Postal Code_x,Healthcare Provider Taxonomy Code_1_x,Healthcare Provider Primary Taxonomy Switch_1_x,Healthcare Provider Taxonomy Code_2_x,Healthcare Provider Primary Taxonomy Switch_2_x,Healthcare Provider Taxonomy Code_3_x,Healthcare Provider Primary Taxonomy Switch_3_x,Healthcare Provider Taxonomy Code_4_x,Healthcare Provider Primary Taxonomy Switch_4_x,Healthcare Provider Taxonomy Code_5_x,Healthcare Provider Primary Taxonomy Switch_5_x,Healthcare Provider Taxonomy Code_6_x,Healthcare Provider Primary Taxonomy Switch_6_x,Healthcare Provider Taxonomy Code_7_x,Healthcare Provider Primary Taxonomy Switch_7_x,Healthcare Provider Taxonomy Code_8_x,Healthcare Provider Primary Taxonomy Switch_8_x,Healthcare Provider Taxonomy Code_9_x,Healthcare Provider Primary Taxonomy Switch_9_x,Healthcare Provider Taxonomy Code_10_x,Healthcare Provider Primary Taxonomy Switch_10_x,Healthcare Provider Taxonomy Code_11_x,Healthcare Provider Primary Taxonomy Switch_11_x,Healthcare Provider Taxonomy Code_12_x,Healthcare Provider Primary Taxonomy Switch_12_x,Healthcare Provider Taxonomy Code_13_x,Healthcare Provider Primary Taxonomy Switch_13_x,Healthcare Provider Taxonomy Code_14_x,Healthcare Provider Primary Taxonomy Switch_14_x,Healthcare Provider Taxonomy Code_15_x,Healthcare Provider Primary Taxonomy Switch_15_x,Taxonomy_Code_x,Classification_x,zipcodes_x,CBSA_x,from_npi_x,to_npi_x,patient_count_x,transaction_count_x,average_day_wait_x,std_day_wait_x,name_x,Entity Type Code_y,Provider Organization Name (Legal Business Name)_y,Provider Last Name (Legal Name)_y,Provider First Name_y,Provider Middle Name_y,Provider Name Prefix Text_y,Provider Name Suffix Text_y,Provider Credential Text_y,Provider First Line Business Practice Location Address_y,Provider Second Line Business Practice Location Address_y,Provider Business Practice Location Address City Name_y,Provider Business Practice Location Address State Name_y,Provider Business Practice Location Address Postal Code_y,Healthcare Provider Taxonomy Code_1_y,Healthcare Provider Primary Taxonomy Switch_1_y,Healthcare Provider Taxonomy Code_2_y,Healthcare Provider Primary Taxonomy Switch_2_y,Healthcare Provider Taxonomy Code_3_y,Healthcare Provider Primary Taxonomy Switch_3_y,Healthcare Provider Taxonomy Code_4_y,Healthcare Provider Primary Taxonomy Switch_4_y,Healthcare Provider Taxonomy Code_5_y,Healthcare Provider Primary Taxonomy Switch_5_y,Healthcare Provider Taxonomy Code_6_y,Healthcare Provider Primary Taxonomy Switch_6_y,Healthcare Provider Taxonomy Code_7_y,Healthcare Provider Primary Taxonomy Switch_7_y,Healthcare Provider Taxonomy Code_8_y,Healthcare Provider Primary Taxonomy Switch_8_y,Healthcare Provider Taxonomy Code_9_y,Healthcare Provider Primary Taxonomy Switch_9_y,Healthcare Provider Taxonomy Code_10_y,Healthcare Provider Primary Taxonomy Switch_10_y,Healthcare Provider Taxonomy Code_11_y,Healthcare Provider Primary Taxonomy Switch_11_y,Healthcare Provider Taxonomy Code_12_y,Healthcare Provider Primary Taxonomy Switch_12_y,Healthcare Provider Taxonomy Code_13_y,Healthcare Provider Primary Taxonomy Switch_13_y,Healthcare Provider Taxonomy Code_14_y,Healthcare Provider Primary Taxonomy Switch_14_y,Healthcare Provider Taxonomy Code_15_y,Healthcare Provider Primary Taxonomy Switch_15_y,Taxonomy_Code_y,Classification_y,zipcodes_y,CBSA_y,from_npi_y,to_npi_y,patient_count_y,transaction_count_y,average_day_wait_y,std_day_wait_y,name_y
0,1275536468,1.0,None,ASAD,MUHAMMAD,None,DR.,None,MD,353 NE

In [141]:
#Adding from provider column
sent_to_vandy_tri_star = sent_to_vandy_tri_star.merge(right = npi_to_name, how = 'left', on = 'NPI')

In [142]:
#Which providers are sent to Tri-Star exclusively
sent_to_vandy_tri_star.groupby('Provider Organization Name (Legal Business Name)_x')['patient_count_x'] \
.sum() \
.reset_index() \
.sort_values(by = 'patient_count_x', ascending = False) \
.reset_index(drop = True)

,Provider Organization Name (Legal Business Name)_x,patient_count_x
0,"HCA HEALTH SERVICES OF TENNESSEE, INC.",3790
1,"CENTENNIAL HOSPITALISTS, LLC",895
2,LONGS PARK EMERGENCY PHYSICIANS PLLC,402
3,LEMMON EMERGENCY PHYSICIANS PLLC,216
4,"LAKESHORE ESTATES, INC.",140
5,INTEGUMETRIX,132
6,"HILLCREST HEALTHCARE, LLC",111
7,"EBENEZER HOME OF TENNESSEE, INC.",103
8,CHRISTIAN CARE CENTER OF CHEATHAM COUNTY INC,90
9,HILLCREST HEALTHCARE LLC,71


In [143]:
#What are the specialties of providers that are sending mass numbers to TriStar
specialties_to_tristar = sent_to_vandy_tri_star.groupby('Classification_x')['patient_count_x'] \
.sum() \
.reset_index() \
.sort_values(by = 'patient_count_x', ascending = False) \
.reset_index(drop = True)

specialties_to_tristar

,Classification_x,patient_count_x
0,Internal Medicine,5148
1,General Acute Care Hospital,3748
2,Orthopaedic Surgery,2853
3,Radiology,2390
4,Nurse Practitioner,1392
5,Physical Therapist,1124
6,Physician Assistant,995
7,"Nurse Anesthetist, Certified Registered",787
8,Thoracic Surgery (Cardiothoracic Vascular Surg...,673
9,Emergency Medicine,658


In [144]:
# Top 10 specialties that are sending patients exclusively to tristar
top_10_specialties_to_tristar = specialties_to_tristar.head(10)

In [145]:
# What orthopedic surgeons or family medicine doctors should Vanderbilt reach to?
sent_to_vandy_tri_star.groupby(['Classification_x','from_provider']) \
['patient_count_x'] \
.sum() \
.reset_index() \
.sort_values(by = 'patient_count_x', ascending = False) \
.reset_index(drop = True) \
.query('Classification_x == "Orthopaedic Surgery" | Classification_x == "Family Medicine"')

,Classification_x,from_provider,patient_count_x
3,Orthopaedic Surgery,DAVID K. DEBOER MD,719
4,Orthopaedic Surgery,MICHAEL J. CHRISTIE MD,709
5,Orthopaedic Surgery,JEFFERSON CRAIG MORRISON MD,635
6,Orthopaedic Surgery,DR.JEFFREY THOMAS HODRICK M.D.,597
27,Orthopaedic Surgery,DR.ROBERT JOEL OTTO M.D.,193
89,Family Medicine,DR.CHRIS MEHR MD,59


In [147]:
# Looking at the different providers that were Internal Medicine
sent_to_vandy_tri_star.groupby(['Classification_x','from_provider','NPI']) \
['patient_count_x'] \
.sum() \
.reset_index() \
.sort_values(by = 'patient_count_x', ascending = False) \
.reset_index(drop = True).head(50) \
.query('Classification_x == "Internal Medicine"')

,Classification_x,from_provider,NPI,patient_count_x
1,Internal Medicine,DR.JOHN ALSTON RIDDICK M.D.,1417131715,1404
2,Internal Medicine,"CENTENNIAL HOSPITALISTS, LLC",1558801332,895
15,Internal Medicine,THOMAS A TESAURO MD,1578501904,270
17,Internal Medicine,JACOB C DEXHEIMER MD,1518014323,266
18,Internal Medicine,VICTOR A VALLEJO MD,1679539886,241
19,Internal Medicine,DR.ROBERT M WHEATLEY M.D. F.A.C.C.,1558461970,236
30,Internal Medicine,JENNIFER P LEE MD,1922048883,189
32,Internal Medicine,THOMAS STEVAN JOHNSTON MD,1023103462,183
38,Internal Medicine,PHILIP S KUO MD,1639112048,144
40,Internal Medicine,JEFFREY C WEBBER MD,1235242181,141


In [ ]:
refer_to

In [150]:
refer_to_tristar

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,name
380,1275536468,1.0,None,ASAD,MUHAMMAD,None,DR.,None,MD,353 NEW SHACKLE ISLAND RD,SUITE 300 C,HENDERSONVILLE,TN,370752379.0,207RI0011X,N,207RC0000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207RC0000X,Internal Medicine,37075,34980,1275536468,1770851552,45,51,32.922,67.310,TRISTAR CARDIOVASCULAR SURGERY LLC
1568,1568464873,2.0,DICKSON MEDICAL ASSOCIATES PC,None,None,None,None,None,None,113 HIGHWAY 70 E,None,DICKSON,TN,370552080.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1568464873,1164834057,68,88,46.807,61.499,"TRISTAR JOINT REPLACEMENT INSTITUTE, LLC"
1759,1568464873,2.0,DICKSON MEDICAL ASSOCIATES PC,None,None,None,None,None,None,113 HIGHWAY 70 E,None,DICKSON,TN,370552080.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1568464873,1609326040,42,65,33.154,37.904,"TRISTAR FAMILY CARE, LLC"
1824,1568464873,2.0,DICKSON MEDICAL ASSOCIATES PC,None,None,None,None,None,None,113 HIGHWAY 70 E,None,DICKSON,TN,370552080.0,207Q00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207Q00000X,Family Medicine,37055,34980,1568464873,1770851552,46,63,12.810,20.686,TRISTAR CARDIOVASCULAR SURGERY LLC
2703,1215932413,2.0,"ANESTHESIA MEDICAL GROUP, PC",None,None,None,None,None,None,110 29TH AVE N,STE 301,NASHVILLE,TN,37203.0,207R00000X,N,207RP1001X,N,2081P2900X,N,208VP0000X,N,367500000X,N,207L00000X,Y,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,207L00000X,Anesthesiology,37203,34980,1215932413,1003152208,274,324,17.222,42.241,"TRISTAR GYNECOLOGY ONCOLOGY, LLC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [152]:
# Adding from provider column to Tristar dataframe
refer_to_tristar = refer_to_tristar.merge(right = npi_to_name, how = 'left', on = 'NPI')

In [159]:
refer_to_tristar.groupby(['Classification','from_provider','NPI']) \
['patient_count'] \
.sum() \
.reset_index() \
.sort_values(by = 'patient_count', ascending = False) \
.reset_index(drop = True).head(50) \
.query('Classification == "Orthopaedic Surgery"')

,Classification,from_provider,NPI,patient_count
12,Orthopaedic Surgery,DAVID K. DEBOER MD,1942265400,719
13,Orthopaedic Surgery,MICHAEL J. CHRISTIE MD,1841255346,709
15,Orthopaedic Surgery,JEFFERSON CRAIG MORRISON MD,1609831916,635
19,Orthopaedic Surgery,DR.JEFFREY THOMAS HODRICK M.D.,1265556674,597


In [168]:
#Finding Kagon's answers
nppes_nashville[nppes_nashville['from_provider'] == 'DICKSON MEDICAL ASSOCIATES PC'].groupby('Classification') \
['patient_count'] \
.sum()

Classification
Family Medicine    91425
Name: patient_count, dtype: int64

In [169]:
nppes_nashville[nppes_nashville['from_provider'] == ''].groupby('Classification') \
['patient_count'] \
.sum()

Series([], Name: patient_count, dtype: int64)

In [170]:
#Can't find James Anderson
nppes_nashville['from_provider'].unique()

array(['MRS.CHERYL DENISE RUDNICKE  CRNP', 'DR.JULIE A PERRIGIN  MD',
       'RICHARD E PRESLEY  M.D.', 'DR.DAVID L ROSS  MD',
       'DR.VENK  MANI  MD', 'DR.JOHN CW MORSE  MD',
       'WILLIAM FLOYD FLEET III M.D.', 'DR.ROBERT W ORGAIN  MD',
       'MELISSA G REYNOLDS  M.D.', 'STACY FAITH DAVIS  M.D.',
       'NATHAN W RUSSELL  FNP', 'TODD GREGORY TOLBERT  M.D.',
       'ELIZABETH L OLDFIELD  M.D.',
       'DARRINGTON PHILLIPS ALTENBERN  M.D.', 'DR.MUHAMMAD  ASAD  MD',
       'DR.GARY T SMITH  MD', 'DR.JOHN R SALYER  MD',
       'ANDREW ROBERTS SAGER  M. D.', 'DR.RICHARD M FLEMING  MD',
       'JACKSON DANIEL COTHREN  MD', 'DR.DAVID J BLAZER  MD',
       'DR.PASCAL A. DAUPHIN  M.D.', 'MR.STEPHEN MATTHEW ROSE  M.D.',
       'LEWIS J BELLARDO  M.D.', 'DR.JUAN S DINKINS  DO',
       'VARALAXMI S DASARI  M.D.', 'KATHERINE CLARKE HANEY  MD',
       'JOHN BRIGHT CAGE  M.D.', 'DR.JEAN K ALEXANDER  MD',
       'WILLIAM OWEN HARTNESS  M.D.', 'ROY P BURCH JR. M.D.',
       'DR.BERNARD C TEKIEL